# user_event 데이터셋 생성 & DB업로드 
* gpt를 통해 데이터셋을 생성하고 DB에 업로드 하는 함수
* 퍼널, 리텐션 등 해당 데이터셋을 통해 sql practice

✅ 데이터 생성 조건
* 이벤트 순서: visit → signup → login → purchase
* 이전 단계 없이 다음 단계 발생 ❌ (e.g. 로그인은 반드시 회원가입 이후)
* 시간 순서 유지: 이벤트 시간이 점진적으로 증가
* 이벤트 수 규칙: 방문자 수 > 회원가입자 수 > 로그인 유저 수 > 구매자 수
* logout 이벤트 제거: 실제 로그아웃 이벤트는 로그로 남지 않는 경우가 많아 제외

In [39]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random

# 시드 고정
random.seed(42)
np.random.seed(42)

# 유저 ID 생성 (500명 방문자)
user_ids = list(range(1001, 1501))

# 퍼널 전환율 설정
signup_rate = 0.8    # 방문자 중 80%가 회원가입
login_rate = 0.7     # 회원가입자 중 70%가 로그인
purchase_rate = 0.5  # 로그인한 유저 중 50%가 구매

# 기본 날짜
start_date = datetime(2025, 1, 1)
max_days = 60

# 속성 정의
channels = ['organic', 'paid_ads', 'social', 'referral']
platforms = ['web', 'mobile']
devices = ['desktop', 'android', 'ios']

# 결과 저장 리스트
rows = []

for user_id in user_ids:
    # 기본 속성
    base_time = start_date + timedelta(
        days=np.random.randint(0, max_days),
        hours=np.random.randint(0, 24),
        minutes=np.random.randint(0, 60)
    )
    channel = random.choice(channels)
    platform = random.choice(platforms)
    device = random.choice(devices)

    # 1. visit 이벤트
    visit_time = base_time
    rows.append([user_id, 'visit', visit_time, channel, platform, device])

    # 2. signup (확률 적용)
    if np.random.rand() < signup_rate:
        signup_time = visit_time + timedelta(minutes=np.random.randint(1, 30))
        rows.append([user_id, 'signup', signup_time, channel, platform, device])

        # 3. login (확률 적용)
        if np.random.rand() < login_rate:
            login_time = signup_time + timedelta(minutes=np.random.randint(1, 30))
            rows.append([user_id, 'login', login_time, channel, platform, device])

            # 4. purchase (확률 적용)
            if np.random.rand() < purchase_rate:
                purchase_time = login_time + timedelta(minutes=np.random.randint(1, 60))
                rows.append([user_id, 'purchase', purchase_time, channel, platform, device])

# DataFrame으로 변환
df = pd.DataFrame(rows, columns=['user_id', 'event_name', 'event_time', 'channel', 'platform', 'device'])

# 시간 순 정렬
df = df.sort_values(by=['user_id', 'event_time'])

# CSV로 저장
df.to_csv('user_events_funnel_clean.csv', index=False)


# DB업로드

In [40]:
import pymysql

conn = pymysql.connect(
         host = 'localhost'
        , port = 3306
        , user = 'root'
        , database = 'sql'
        , password = '1234'
        , charset = 'utf8mb4'
        )
cursor = conn.cursor()

# db테이블 생성
table_name = 'user_event'
create_sql = f"""create table if not exists `{table_name}` (
                user_id INT,
                event_name VARCHAR(255),
                event_time DATETIME,
                channel VARCHAR(255),
                platform VARCHAR(255), 
                device VARCHAR(255)
                )
                """

cursor.execute(create_sql)

# DB업로드
columns = ', '.join(df.columns)
placeholders = ', '.join(['%s'] * len(df.columns))

# query
sql = f"insert into {table_name} ({columns}) values ({placeholders})"

# insert 
data = [tuple(row) for row in df.to_numpy()]

cursor.executemany(sql, data)
conn.commit()